# Snake Classifier with AlexNet

## Imports

In [90]:
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import keras
import numpy as np
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(1000)

## Reading in Data

In [97]:
non_venomous_dir = "./Non-Venomous"
venomous_dir = "./Venomous"
snake_data = {}
label_dict = {}
target_size = (224,224,3)
#counter for species id
i = 0

for species in os.listdir(non_venomous_dir):
    venomous_species.append(species)
    img_list = []
    species_folder = os.listdir(os.path.join(non_venomous_dir,species))
    #print(species_folder)
    label_dict[species] = i
    for img_name in species_folder:
        img = load_img(non_venomous_dir + "/" + species + "/" + img_name)
        img = img.resize((224,224))
        img_array = img_to_array(img)
        img_list.append(img_array)
        img_label
        
    snake_data[species] = np.array(img_list)
    i += 1
    
    
    
for species in os.listdir(venomous_dir):
    venomous_species.append(species)
    img_list = []
    species_folder = os.listdir(os.path.join(venomous_dir,species))
    label_dict[species] = i
    for img_name in species_folder:
        img = load_img(venomous_dir + "/" + species + "/" + img_name)
        img = img.resize((224,224))
        img_array = img_to_array(img)
        img_list.append(img_array)
        
    snake_data[species] = np.array(img_list)
    i += 1

    

## Organizing Data for Training

In [102]:
img_array = []
img_labels = []
for species in snake_data:
    for img in snake_data[species]:
        img_array.append(img)
        img_labels.append(label_dict[species])
img_array = np.array(img_array)
img_labels = np.array(img_labels)

In [104]:
img_array,img_labels = shuffle(img_array,img_labels)

In [116]:
train_percent = .8
train_count = int(train_percent*len(img_array))
test_count = len(img_array) - train_count

train_labels = img_labels[0:train_count]
test_labels = img_labels[train_count:]

train_images = img_array[0:train_count]
test_images = img_array[train_count:]

train_images = train_images / 255.0

test_images = test_images / 255.0

In [117]:
num_category = len(snake_data.keys())

# convert class vectors to binary class matrices
train_labels = keras.utils.to_categorical(train_labels, num_category)
test_labels = keras.utils.to_categorical(test_labels, num_category)

In [118]:
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(num_category))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_28 (Activation)   (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_29 (Activation)   (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 6, 6, 384)         885120    
__________

In [119]:
#conv_model.fit(train_images,train_labels,epochs=5)
batch_size = 1
num_epoch = 10
#model training
model_log = model.fit(train_images,train_labels,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(test_images,test_labels))

Train on 1423 samples, validate on 356 samples
Epoch 1/10
1423/1423 [==============================] - 218s 154ms/step - loss: 2.4422 - acc: 0.1455 - val_loss: 2.4308 - val_acc: 0.1208
Epoch 2/10
1423/1423 [==============================] - 219s 154ms/step - loss: 2.4191 - acc: 0.1363 - val_loss: 2.4254 - val_acc: 0.1348
Epoch 3/10
1423/1423 [==============================] - 212s 149ms/step - loss: 2.4169 - acc: 0.1469 - val_loss: 2.4216 - val_acc: 0.1348
Epoch 4/10
1423/1423 [==============================] - 221s 155ms/step - loss: 2.4175 - acc: 0.1363 - val_loss: 2.4246 - val_acc: 0.1208
Epoch 5/10
1423/1423 [==============================] - 214s 151ms/step - loss: 2.4142 - acc: 0.1434 - val_loss: 2.4268 - val_acc: 0.1348
Epoch 6/10
1423/1423 [==============================] - 211s 148ms/step - loss: 2.4081 - acc: 0.1342 - val_loss: 2.4242 - val_acc: 0.1348
Epoch 7/10
1423/1423 [==============================] - 214s 150ms/step - loss: 2.4116 - acc: 0.1441 - val_loss: 2.4241 - val